#PROYECTO MÁSTER KSCHOOL FASE 1: LIMPIEZA

**@ GUILLEM ROCHINA & HELENA SAIGÍ**

## Los datos

El dataset que se limpiará a continuacion procede de la web [insideairbnb.com](http://insideairbnb.com/). Consiste en datos 'scrapeados' de la web oficial de AirBNB, donde podemos encontrar archivos csv con información detalla de los pisos ofertados en la web **(listings)**, así como el registro diario de precios **(calendar)** de la ciudad de Barcelona.

El objetivo de este notebook es el de recopilar, analizar y detectar los datos útiles para generar un dataset base para la posterior exploración y elaboración de modelos predictivos. Para ello haremos uso de los datasets de listings y calendar, centrando la mayor parte del proceso de limpieza en el primero. 

**Importación de librerías y datos**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 100) # Para evitar que nos corte los prints

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Listings/April2017.zip')
df = df.append(pd.read_csv('https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Listings/April2018.zip'), ignore_index = True)
df = df.append(pd.read_csv('https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Listings/April2019.zip'), ignore_index = True)
df = df.append(pd.read_csv('https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Listings/April2020.zip'), ignore_index = True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##**Listings**

## Limpieza Superficial

En primer lugar realizaremos una limpieza de los elementos que debido su naturaleza, no nos sirven para el análisis de nuestro proyecto. Por ejemplo, columnas que recopilan urls y que no son de utilidad. Por otro lado, dado que a lo largo de este proyeto se ha decidido no hacer uso de NLP para centrarse en otros puntos del análisis, las columnas de texto serán eliminadas también junto a las anteriormente mencionadas.

In [ ]:
df.drop(df.columns[df.columns.str.endswith('url')], axis = 1, inplace = True)

In [ ]:
DropC = ['name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit',
         'interaction', 'house_rules', 'access', 'jurisdiction_names']
df.drop(DropC, axis = 1, inplace = True)

Como podemos observar, de los **75558** listings que posee nuestro dataset, **32046** son pisos repetidos, por lo cual eliminaremos los duplicamos a fin de reducir el coste computacional de nuestro posterior código.

In [ ]:
print(df.shape[0])
print(df.shape[0]-len(df['id'].unique()))

75558
32046


In [ ]:
df.drop_duplicates(subset = ['id'], inplace = True)

A continuación, eliminaremos las columnas cuyo porcentaje de **nulls** supere el 60%. Debido a que las columnas con dicho porcentaje no nos resultan de gran utilidad.

In [ ]:
nulls = df.isnull().sum() / df.shape[0]
nulls[nulls>0.5]

host_acceptance_rate                            0.825129
square_feet                                     0.978810
weekly_price                                    0.924527
monthly_price                                   0.921677
license                                         0.615141
minimum_minimum_nights                          0.620771
maximum_minimum_nights                          0.620771
minimum_maximum_nights                          0.620771
maximum_maximum_nights                          0.620771
minimum_nights_avg_ntm                          0.620771
maximum_nights_avg_ntm                          0.620771
number_of_reviews_ltm                           0.620771
calculated_host_listings_count_entire_homes     0.620771
calculated_host_listings_count_private_rooms    0.620771
calculated_host_listings_count_shared_rooms     0.620771
dtype: float64

In [ ]:
df.drop(nulls[nulls>0.6].index, axis = 1, inplace = True)

Por otro lado, columnas respecto al scraping o información del host no resulta de gran interés, por ello, las eliminamos también del dataset.

In [ ]:
DropC = ['scrape_id', 'last_scraped', 'host_name', 'host_location', 'host_neighbourhood', 'calendar_updated', 'calendar_last_scraped'] # Aunque parezca redundante hacer esto, ayuda a una mejor lectura del código.
df.drop(DropC, axis = 1, inplace = True)

##Limpieza Exhaustiva

Una vez eliminadas las columnas menos útiles a primera vista, procedemos a analizar más en profundidad el resto de columnas. **Empezaremos con las que contengan strings (de tipo object).**

In [ ]:
objectcol = df.dtypes[df.dtypes == 'object']
numcol = df.dtypes[df.dtypes != 'object']
objectcol

experiences_offered                 object
host_since                          object
host_about                          object
host_response_time                  object
host_response_rate                  object
host_is_superhost                   object
host_verifications                  object
host_has_profile_pic                object
host_identity_verified              object
street                              object
neighbourhood                       object
neighbourhood_cleansed              object
neighbourhood_group_cleansed        object
city                                object
state                               object
zipcode                             object
market                              object
smart_location                      object
country_code                        object
country                             object
is_location_exact                   object
property_type                       object
room_type                           object
bed_type   

In [ ]:
df['experiences_offered'].sample(20)

42413    none
73578    none
13410    none
24239    none
68786    none
74673    none
73182    none
54098    none
59080    none
19097    none
52647    none
6684     none
35785    none
16416    none
10831    none
36425    none
64339    none
31241    none
160      none
48604    none
Name: experiences_offered, dtype: object

### Experiences offered

Dicha columna suponemos que recoge las experiencias que los propietarios ofrecen a sus posibles inquilinos. No obstante no podemos sacar conclusiones fiables debido a la poca información que proporciona.

In [ ]:
display(df['experiences_offered'].value_counts())
df.drop(['experiences_offered'], axis = 1, inplace = True)

none    43512
Name: experiences_offered, dtype: int64

Ni uno de los propietarios ha publicado nada en esta sección de su oferta y sin embargo, no se han reconocido como **NaN** debido a que se ha introducido **none** en vez de dejarlo en blanco, por lo que, una vez detectado, es innecesario mantener esta columna.

### Host_since

Fecha en la que el propietario se registró en la plataforma de AirBNB

La columna **host_since** nos servirá en un futuro (en la fase de Exploración) para calcular la antigüedad del propietario en AirBNB (Creemos que los usuarios más veteranos podrían poner precios de alquiler más altos).

In [ ]:
df['host_since']

0        2013-07-28
1        2015-11-11
2        2015-08-08
3        2015-10-31
4        2013-07-18
            ...    
75553    2012-08-08
75554    2013-02-06
75555    2019-01-25
75556    2013-02-06
75557    2017-04-30
Name: host_since, Length: 43512, dtype: object

### Host_response_time

La columna host response time nos indica el tiempo que tarda de media el propietario en responder a las cuestiones o mensajes que sus inquilinos o posibles inclinos realizan.

La existencia de 5627 registros con **NaNs** en la columna de **host_response_time** puede deberse a la incapacidad de determinar cuanto tarda el propietario en responder a los mensajes, no obstante, eliminar las filas sería un error ya que el resto de información de los listings parece correcta, por lo que con el objetivo de mantener dichas filas, se sustituyen todos los valores faltantes por una nueva categoría, **undetemined** (para mantener la consistencia del idioma).

In [ ]:
df['host_response_time'].value_counts(dropna = False)

within an hour        23711
within a few hours     7546
NaN                    5627
within a day           5231
a few days or more     1397
Name: host_response_time, dtype: int64

In [ ]:
df['host_response_time'].fillna('undetermined', inplace = True)
df['host_response_time'].value_counts(dropna = False)

within an hour        23711
within a few hours     7546
undetermined           5627
within a day           5231
a few days or more     1397
Name: host_response_time, dtype: int64

### Host_response_rate

Host_response_rate indica el porcentaje de preguntas y mensajes que los propietarios responden normalmente.

Como podemos observar, la mayoría de propietarios suele responder la mayoría de los mensajes de los usuarios (el 80% de propietarios responden a más del 80% de mensajes). Por otro lado, existe un 13% de **Nulls** que sospechamos es debido a su reciente incorporación a la web (No ha dado tiempo a registrar esos valores debido a su corto periodo en la plataforma).

Por tanto, a pesar de que se podría recurrir a la discretización de esta variable, hemos optado por eliminarla, ya que sospechamos que nos va a aportar poca información y queremos evitar en la medida de lo posible la **maldición de la dimensionalidad**.

In [ ]:
temp = df['host_response_rate'].str.replace('%', '').fillna('9999').astype('int')
temp.value_counts(dropna = False, normalize = True).sort_index().cumsum()[-25:]

77      0.088160
78      0.089883
79      0.092021
80      0.110820
81      0.112567
82      0.115233
83      0.123115
84      0.123759
85      0.125896
86      0.131044
87      0.133825
88      0.138743
89      0.145178
90      0.177078
91      0.182938
92      0.188477
93      0.196566
94      0.206817
95      0.216055
96      0.226145
97      0.243910
98      0.261468
99      0.290977
100     0.870679
9999    1.000000
Name: host_response_rate, dtype: float64

In [ ]:
df.drop('host_response_rate', axis = 1, inplace = True)

### Host_is_superhost

Ser superhost implica cierta veteranía en AirBNB, por lo que se podrían registrar precios más altos entre los listings de usuarios que disfrutan de esta situación.

El atributo **host_is_superhost** se trata de una variable dicotómica (True o False), por lo que a fin de tener consistencia con los algoritmos de **Scikit-Learn** realizamos el llamado *One Hot Encoding* a fin de crear un variable dummy.

In [ ]:
df['host_is_superhost'].value_counts()

f    39875
t     3621
Name: host_is_superhost, dtype: int64

In [ ]:
df['host_is_superhost'] = df['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

### Host_verifications

La columna host_verifications indica que formas de contacto e identificación aportados por el propietario han sido verificadas por AirBNB.

Las verificaciones de los propietarios las hemos tratado mediante la creación de variables dummy en los que queda registrado el propietario posee (1) o no (0) dicha verificación. Existen muchas más verificaciones de las que hemos añadido, pero a fin de no crear excesivos atributos, hemos optado por las 5 que podéis ver en el siguiente código.

In [ ]:
temp = df['host_verifications'].str.replace('[','').str.replace(']','').str.replace("'", "")
temp.value_counts(normalize = True).sort_values(ascending = False)[0:10]

email, phone, reviews                                                                          0.169241
email, phone                                                                                   0.121828
email, phone, reviews, jumio                                                                   0.114474
email, phone, reviews, jumio, government_id                                                    0.057203
email, phone, facebook, reviews                                                                0.041414
email, phone, reviews, jumio, offline_government_id, selfie, government_id, identity_manual    0.040862
email, phone, jumio, offline_government_id, selfie, government_id, identity_manual             0.039208
email, phone, facebook, reviews, jumio                                                         0.030566
phone                                                                                          0.029302
email, phone, reviews, jumio, offline_government_id, government_

In [ ]:
for verif in ['email', 'phone', 'reviews', 'jumio', 'government_id']:
  print('\nEl {}% de los propietarios tienen {} verificado'.format(temp.str.contains(verif).sum()/temp.shape[0], verif))


El 0.9329840044125759% de los propietarios tienen email verificado

El 0.992186063614635% de los propietarios tienen phone verificado

El 0.7023349880492737% de los propietarios tienen reviews verificado

El 0.5191441441441441% de los propietarios tienen jumio verificado

El 0.37226512226512226% de los propietarios tienen government_id verificado


In [ ]:
df['host_emailverified'] = df['host_verifications'].apply(lambda x: 1 if 'email' in x else 0)
df['host_phoneverified'] = df['host_verifications'].apply(lambda x: 1 if 'phone' in x else 0)
df['host_hasjumio'] = df['host_verifications'].apply(lambda x: 1 if 'jumio' in x else 0)
df['host_reviewverified'] = df['host_verifications'].apply(lambda x: 1 if 'review' in x else 0)
df['host_selfieverified'] = df['host_verifications'].apply(lambda x: 1 if 'selfie' in x else 0)
df['host_idverified'] = df['host_verifications'].apply(lambda x: 1 if 'government_id' in x else 0)
df.drop(['host_verifications'], axis = 1, inplace = True)

### Host_has_profile_pic y Host_identity_verified

Ambas columnas indican con True or False las condiciones estipuladas en sus nombres, si el propietario tiene foto de perfil y si su identidad ha sido verificada por AriBNB. Las otras dos variables son dicotómicas, por lo que aplicamos el mismo procedimiento de antes.

In [ ]:
df['host_has_profile_pic'] = df['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
df['host_identity_verified'] = df['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)

### Columnas relacionadas con la localización de los alojamientos

Mientras que street es una columna que nos aporta poca información (todos los pisos son de Barcelona), las tres columnas referidas al barrio indican lo mismo, por lo que nos quedamos con la más resumida de todas, **neighbourhood_group_cleansed**.

In [ ]:
df[['street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed']]

,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed
0,"El Raval, Barcelona, Catalunya 08001, Spain",El Raval,el Raval,Ciutat Vella
1,"Barcelona, Catalunya 08005, Spain",NaN,el Raval,Ciutat Vella
2,"El Raval, Barcelona, Catalunya 08001, Spain",El Raval,el Raval,Ciutat Vella
3,"El Raval, Barcelona, Catalunya 08001, Spain",El Raval,el Raval,Ciutat Vella
4,"El Raval, Barcelona, Catalonia 08001, Spain",El Raval,el Raval,Ciutat Vella
...,...,...,...,...
75553,"Barcelona, Catalunya, Spain",Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",Ciutat Vella
75554,"Barcelona, Catalunya, Spain",Dreta de l'Eixample,la Dreta de l'Eixample,Eixample
75555,"Barcelona, Catalunha, Spain",Eixample,Sant Antoni,Eixample
75556,"Barcelona, Catalunya, Spain",Ciutat Vella,el Raval,Ciutat Vella


In [ ]:
DropC = ['street', 'neighbourhood', 'neighbourhood_cleansed']
df.drop(DropC, axis = 1, inplace = True)

In [ ]:
df['neighbourhood_group_cleansed'].value_counts(dropna = False)

Eixample               13517
Ciutat Vella           10802
Sants-Montjuïc          5155
Sant Martí              4731
Gràcia                  3730
Sarrià-Sant Gervasi     1689
Horta-Guinardó          1423
Les Corts               1081
Sant Andreu              798
Nou Barris               586
Name: neighbourhood_group_cleansed, dtype: int64

El resto de columnas referidas a la localización de listings nos aportan información muy poco útil (es evidente la localización de los listings si todos son de Barcelona) por lo que las eliminamos sin mayor problema.

In [ ]:
display(df[['city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country']])
DropC = ['city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country']
df.drop(DropC, axis = 1, inplace = True)

,city,state,zipcode,market,smart_location,country_code,country
0,Barcelona,Catalunya,08001,Barcelona,"Barcelona, Spain",ES,Spain
1,Barcelona,Catalunya,08005,Barcelona,"Barcelona, Spain",ES,Spain
2,Barcelona,Catalunya,08001,Barcelona,"Barcelona, Spain",ES,Spain
3,Barcelona,Catalunya,08001,Barcelona,"Barcelona, Spain",ES,Spain
4,Barcelona,Catalonia,08001,Barcelona,"Barcelona, Spain",ES,Spain
...,...,...,...,...,...,...,...
75553,Barcelona,Catalunya,8003,Barcelona,"Barcelona, Spain",ES,Spain
75554,Barcelona,Catalunya,8010,Barcelona,"Barcelona, Spain",ES,Spain
75555,Barcelona,Catalunha,8011,Barcelona,"Barcelona, Spain",ES,Spain
75556,Barcelona,Catalunya,8001,Barcelona,"Barcelona, Spain",ES,Spain


### Is_location_exact

Nos indica si la localización se específica concretamente o si se dan indicaciones vagas respecto a el paradero del alojamiento. Aunque dudamos que este atributo sea útil lo trataremos como una variable dicotómica más y analizaremos más adelante si nos puede ser de valor o no.

In [ ]:
df['is_location_exact'].value_counts()

t    28547
f    14965
Name: is_location_exact, dtype: int64

In [ ]:
df['is_location_exact'] = df['is_location_exact'].apply(lambda x: 1 if x == 't' else 0)

### Property_type

Al ser categorías muy similares entre sí, generalizaremos esta columna con cuatro categorías (**Apartment, House, Hotel y Other**), ya que queremos evitar la maldición de la dimensionalidad a la vez que mantenemos todos los registros intactos.

In [ ]:
df['property_type'].value_counts()

Apartment                 37267
House                      1109
Loft                        998
Condominium                 831
Serviced apartment          678
Bed & Breakfast             440
Guest suite                 339
Other                       327
Hostel                      280
Bed and breakfast           259
Boutique hotel              200
Guesthouse                  149
Boat                        141
Hotel                       108
Dorm                         81
Casa particular (Cuba)       58
Aparthotel                   55
Townhouse                    48
Villa                        39
Camper/RV                    17
Vacation home                15
In-law                        9
Dome house                    8
Chalet                        8
Tiny house                    6
Timeshare                     6
Barn                          5
Casa particular               5
Earth house                   3
Cottage                       3
Cave                          3
Cabin   

In [ ]:
tempdict = {'Loft':'Apartment', 'Condominium':'House', 'Serviced apartment': 'Apartment', 'Bed & Breakfast':'Hotel', 'Guest suite': 'Hotel', 'Hostel': 'Hotel', 'Dorm': 'Hotel', 'Casa particular (Cuba)':'House', 'Aparthotel': 'Hotel',
            'Townhouse': 'House', 'Villa': 'House', 'Vaction home': 'House', 'Chalet': 'House', 'Dome house': 'House', 'Tiny house': 'House', 'Casa Particular': 'House', 'Apartment': 'Apartment', 'Hotel':'Hotel', 'House':'House'}
df['property_type'] = df['property_type'].map(tempdict).fillna('Other')
df['property_type'].value_counts()

Apartment    38943
House         2107
Hotel         1303
Other         1159
Name: property_type, dtype: int64

### Room_type

Indica el tipo de Habitación de la que dispone el alojamiento. No se realizará ningún tratamiento sobre este atributo.

In [ ]:
df['room_type'].value_counts()

Private room       22841
Entire home/apt    20066
Shared room          493
Hotel room           112
Name: room_type, dtype: int64

### Bed_type

Resgistra el tipo de cama de la que dispone el alojamiento. Debido a que la mayoría de registros incluyen una *Real Bed*, eliminamos está columna ya que no nos aporta demasiada información.

In [ ]:
display(df['bed_type'].value_counts())
df.drop(['bed_type'], axis = 1, inplace = True)

Real Bed         43188
Pull-out Sofa      219
Futon               81
Couch               15
Airbed               8
Name: bed_type, dtype: int64

### Amenities

La columna **Amenities**, como su nombre indica, incluye por listing una lista de servicios adicionales al alojamiento que el propietario pone al servicio de huésped.

In [ ]:
amenities = df['amenities'].str.replace('{', '').str.replace('}', '').str\
                          .replace('"', '').str.split(",")
dict_amen= {}
for lista in amenities:
    for elemento in lista:
        if elemento in dict_amen:
            dict_amen[elemento] += 1
        else:
            dict_amen[elemento] = 1

ser_amen = pd.Series(dict_amen, index = dict_amen.keys())
(ser_amen[ser_amen>df.shape[0]*.10]/df.shape[0]).sort_values(ascending = False)

Essentials                                    0.900648
Kitchen                                       0.888927
Washer                                        0.760480
Heating                                       0.734717
Hangers                                       0.731361
Hair dryer                                    0.670757
TV                                            0.668046
Iron                                          0.628401
Shampoo                                       0.587838
Wifi                                          0.574370
Air conditioning                              0.523396
Laptop friendly workspace                     0.491657
Family/kid friendly                           0.393317
Wireless Internet                             0.386192
Elevator                                      0.320279
Hot water                                     0.319061
Internet                                      0.300446
Smoking allowed                               0.264938
Elevator i

Como podemos observar, hay ciertas comodidades que son muy comunes entre los registros, como la Cocina **(Kitchen)** o Perchas **(Hangers)**, así como otras muy poco comunes que caen por debajo del 10% de apariciones y han sido ignoradas. Tras cierto análisis, se ha decidido utilizar tan solo las comodidades que se han considerado medianamente comunes o poco comunes, y que debido al valor añadido que aportan, consideramos relevantes en nuestro análisis, como la existencia de Aire Acondicionado o Ascensor en el alojamiento.

In [ ]:
columnselection = ['Air conditioning', 'Family/kid friendly', 'Host greets you', 'Laptop friendly workspace', 'Paid parking off premises', 
                  'Patio or balcony', 'Luggage dropoff allowed', 'Long term stays allowed', 'Smoking allowed', 'Step-free access',
                  'Pets allowed', '24-hour check-in']
for column in columnselection:
    df[column] = df['amenities'].apply(lambda x: 1 if column in x else 0)
df['Elevator'] = df['amenities'].apply(lambda x: 1 if ('Elevator' in x or 'Elevator in building' in x) else 0)
df.drop('amenities', axis = 1, inplace = True)

### Columnas relacionadas con precios

Las columnas relacionadas con precios se tratarán todas de la misma forma a través de 'method chaining' que eliminará las comas y los símbolos de dolar para posteriormente transformar los datos a **float**.

**price**: Indica el precio al que se oferta el alojamiento. Esta columna será eliminada más adelante cuando se incluyan los precios registrados en el calendar.

**security_deposit**: Indica la cantidad de deposito que se debe abonar para reservar el alojamiento, los **Nulls** han sido tratados como 0 debido a que suponemos que el no indicar cantidad precisamente indica que no se debe abonar ninguna cantidad.

**cleaning_fee**: Tasa que se debe pagar para cubrir los costes de limpieza. El tratamiento de los **Nulls** se aborda igual que el security_deposit.

**extra_people**: Registra el pago que se debe realizar en caso de añadir una persona más de la especificada en el listing. El tratamiento de los **Nulls** se aborda igual que el security_deposit.

In [ ]:
df[['price', 'security_deposit', 'cleaning_fee', 'extra_people']].sample(5)

,price,security_deposit,cleaning_fee,extra_people
71176,$40.00,$0.00,$35.00,$0.00
71142,$65.00,NaN,NaN,$0.00
35822,$20.00,$0.00,$0.00,$15.00
35909,$250.00,$200.00,$50.00,$0.00
16064,$380.00,NaN,NaN,$20.00


In [ ]:
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype('float')

In [ ]:
df['security_deposit'] = df['security_deposit'].str.replace('$', '').str.replace(',','').fillna(0).astype('float')

In [ ]:
df['cleaning_fee'] = df['cleaning_fee'].str.replace('$', '').str.replace(',','').fillna(0).astype('float')

In [ ]:
df['extra_people'] = df['extra_people'].str.replace('$', '').str.replace(',','').fillna(0).astype('float')

In [ ]:
df[['price', 'security_deposit', 'cleaning_fee', 'extra_people']].sample(5)

,price,security_deposit,cleaning_fee,extra_people
28337,25.0,0.0,5.0,0.0
14836,75.0,0.0,40.0,6.0
53593,39.0,0.0,0.0,0.0
31741,50.0,0.0,30.0,0.0
20752,60.0,0.0,0.0,0.0


### Has_availability

Columna que aporta poca información, ya que todas los listings indican que tienen disponibilidad y los que no indican son Nulls, por lo que la eliminamos.

In [ ]:
display(df['has_availability'].value_counts(dropna = False))
df.drop(['has_availability'], axis = 1, inplace = True)

t      25859
NaN    17653
Name: has_availability, dtype: int64

### First_review y Last_review

Indica las fechas en las que se hizo la primera y la última review

Aunque las fechas en las que se realizaron la primera y la última review podría aportar información útil para nuestro análisis, encontramos un 29% de nulls en los registros. El hecho de imputar valores implicaría introducir un sesgo muy grande al ser casi 1/3 de los elementos, por lo que hemos optado por ignorar estas columnas y arriesgarnos a perder un poco de información.

In [ ]:
df[['first_review', 'last_review']].head(5)

,first_review,last_review
0,2015-09-25,2015-09-27
1,2016-09-22,2017-02-09
2,2015-08-19,2015-11-16
3,2015-11-06,2017-03-19
4,2013-08-08,2017-03-28


In [ ]:
display(df['last_review'].isnull().sum()/df.shape[0])
display(df['last_review'].isnull().sum()/df.shape[0])
df.drop(['first_review', 'last_review'], axis = 1, inplace = True)

0.28978212906784334

0.28978212906784334

### Cancellation_policy

Columna categórica que registra el tipo de política de cancelaciones que cada propietario lleva a cabo. Como observamos, hay muchas categorías con pocos registros, por lo que, a fin de reducir el número de atributos, vamos a realizar una generalización en 4 categorías basandonos en la similaridad de estas.


In [ ]:
df['cancellation_policy'].value_counts()

flexible                       12927
strict                         11880
moderate                       10529
strict_14_with_grace_period     7632
super_strict_30                  493
super_strict_60                   49
luxury_moderate                    2
Name: cancellation_policy, dtype: int64

In [ ]:
tempdict = {'strict_14_with_grace_period': 'strict_less30', 'flexible':'flexible', 
            'moderate':'moderate', 'luxury_moderate': 'moderate', 
            'super_strict_30':'strict_30orMore', 'super_strict_60':'strict_30orMore', 
            'strict':'strict_less30'}
df['cancellation_policy'] = df['cancellation_policy'].map(tempdict)

In [ ]:
df['cancellation_policy'].value_counts()

strict_less30      19512
flexible           12927
moderate           10531
strict_30orMore      542
Name: cancellation_policy, dtype: int64

### Require_guest_profile_picture, Require_guest_phone_verification y Is_business_travel_ready

Columnas poco relevantes debido a que la mayoría de registros se concentran en una categoría. Por lo que las eliminamos.

In [ ]:
display(df['require_guest_profile_picture'].value_counts(dropna = False))
df.drop(['require_guest_profile_picture'], axis = 1, inplace = True)

f    42885
t      627
Name: require_guest_profile_picture, dtype: int64

In [ ]:
display(df['require_guest_phone_verification'].value_counts(dropna = False))
df.drop(['require_guest_phone_verification'], axis = 1, inplace = True)

f    42233
t     1279
Name: require_guest_phone_verification, dtype: int64

In [ ]:
display(df['is_business_travel_ready'].value_counts(dropna = False))
df.drop(['is_business_travel_ready'], axis = 1, inplace = True)

f      25796
NaN    17653
t         63
Name: is_business_travel_ready, dtype: int64

**Ahora pasamos a las columnas numéricas**

In [ ]:
numcol

id                                  int64
host_id                             int64
host_listings_count               float64
host_total_listings_count         float64
latitude                          float64
longitude                         float64
accommodates                        int64
bathrooms                         float64
bedrooms                          float64
beds                              float64
guests_included                     int64
minimum_nights                      int64
maximum_nights                      int64
availability_30                     int64
availability_60                     int64
availability_90                     int64
availability_365                    int64
number_of_reviews                   int64
review_scores_rating              float64
review_scores_accuracy            float64
review_scores_cleanliness         float64
review_scores_checkin             float64
review_scores_communication       float64
review_scores_location            

In [ ]:
df[numcol.index].isnull().sum()

id                                    0
host_id                               0
host_listings_count                  16
host_total_listings_count            16
latitude                              0
longitude                             0
accommodates                          0
bathrooms                            56
bedrooms                             47
beds                                398
guests_included                       0
minimum_nights                        0
maximum_nights                        0
availability_30                       0
availability_60                       0
availability_90                       0
availability_365                      0
number_of_reviews                     0
review_scores_rating              13245
review_scores_accuracy            13283
review_scores_cleanliness         13268
review_scores_checkin             13296
review_scores_communication       13275
review_scores_location            13295
review_scores_value               13304


### Columnas de Ids

Conservamos las columna de **id** para futuros joins, pero eliminamos la columna de **host_id**.

In [ ]:
df[['id', 'host_id']].head(5)

,id,host_id
0,8207551,7783989
1,14958299,48792562
2,7766152,40843803
3,9237132,47842001
4,1406827,7573380


In [ ]:
df.drop(['host_id'], axis = 1, inplace = True)

### Host_listings_count

Indica el número de alojamientos que ofrece un propietario en AirBNB

In [ ]:
df[['host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count']].sort_values(by = 'host_total_listings_count', ascending = False)

,host_listings_count,host_total_listings_count,calculated_host_listings_count
52002,1020.0,1020.0,1
70014,599.0,599.0,3
70015,599.0,599.0,3
70016,599.0,599.0,3
18951,512.0,512.0,127
...,...,...,...
33071,NaN,NaN,1
34923,NaN,NaN,1
36632,NaN,NaN,1
37890,NaN,NaN,1


Las columnas de los **listings_count** son en su mayoría coincidentes, por lo que nos quedaremos tan solo con **listings_count**

In [ ]:
(df['host_listings_count'] == df['host_total_listings_count']).value_counts(normalize = True)

True     0.999632
False    0.000368
dtype: float64

In [ ]:
(df['host_listings_count'] == df['calculated_host_listings_count']).value_counts(normalize = True)

True     0.6996
False    0.3004
dtype: float64

In [ ]:
df.drop(['host_total_listings_count'], axis = 1, inplace = True)
df.drop(['calculated_host_listings_count'], axis = 1, inplace = True)
df.dropna(subset = ['host_listings_count'], inplace = True)

### Accommodates

Indica el número de huéspedes que pueden instalarse en el alojamiento. No se realizará ningún tratamiento de esta variable.

In [ ]:
df['accommodates'].value_counts(dropna = False)

2     17495
4      7535
1      6055
6      3737
3      3521
5      2431
8      1147
7       652
10      347
9       200
16      134
12      123
14       50
11       39
13       17
15       13
Name: accommodates, dtype: int64

### Bathrooms, Bedrooms y Beds

Registra el número de Baños, Dormitorios y Camas disponibles en el alojamiento.

In [ ]:
df[['bathrooms', 'bedrooms', 'beds']].head(5)

,bathrooms,bedrooms,beds
0,1.0,1.0,1.0
1,1.0,1.0,1.0
2,1.0,0.0,1.0
3,1.0,2.0,2.0
4,1.0,1.0,4.0


Como encontramos ciertos **NaNs** en los registros, hemos decidido realizar una imputación de medianas en los valores no registrados. Además, con el objetivo de que quede constacia (como medida preventiva) de la imputación, creamos una columna donde quede grabado en que registros se han imputado los valores.

In [ ]:
df[['bathrooms', 'bedrooms', 'beds']].isnull().sum()

bathrooms     56
bedrooms      47
beds         398
dtype: int64

In [ ]:
df['bathrooms_imput'] = df['bathrooms'].isnull().astype('int')
df['bathrooms'].fillna(df['bathrooms'].median(), inplace = True)

In [ ]:
df['bedrooms_imput'] = df['bedrooms'].isnull().astype('int')
df['bedrooms'].fillna(df['bedrooms'].median(), inplace = True)

In [ ]:
df['beds_imput'] = df['beds'].isnull().astype('int')
df['beds'].fillna(df['beds'].median(), inplace = True)

### Guest_included

Número de huéspedes que se incluyen en la reserva (sin incluir los adicionales, si los hubiera). No se realizará tratamiento de esta variable.

In [ ]:
df[['guests_included']].head(5)

,guests_included
0,1
1,1
2,1
3,2
4,2


### Minimum_nights y Maximum_nights

Número mínimo y máximo de noches relacionados con la reserva del alojamiento. No se realizará tratamiento de esta variable.

In [ ]:
df[['minimum_nights', 'maximum_nights']].head(5)

,minimum_nights,maximum_nights
0,1,1125
1,1,1125
2,2,30
3,2,1125
4,2,30


### Availability_{}

Las columnas de availability incluyen el número de días que el alojamiento está disponible en los próximos 30, 60, 90 y 365

In [ ]:
df[['availability_30', 'availability_60', 'availability_90', 'availability_365']].head(5) 

,availability_30,availability_60,availability_90,availability_365
0,0,0,0,0
1,0,0,0,99
2,0,0,0,151
3,2,14,39,303
4,1,5,8,26


Como hemos eliminado los id repetidos y se ha quedao el último registro disponible por id, no tiene sentido utilizar las availabilities más allá de la de 365 (si más adelante utilizamos la availability diaria de calendar esta tampoco será demasiado relevante), por lo que solo conservaremos la columna que incluye los 365 días.

In [ ]:
df.drop(['availability_30', 'availability_60', 'availability_90'], axis = 1, inplace = True)

### Reviews per month y Number_of_reviews

In [ ]:
df[['number_of_reviews', 'reviews_per_month']].head(5)

,number_of_reviews,reviews_per_month
0,2,0.11
1,15,2.26
2,8,0.40
3,55,3.17
4,226,5.06


In [ ]:
df[['number_of_reviews', 'reviews_per_month']].isnull().sum() / df.shape[0]

number_of_reviews    0.000000
reviews_per_month    0.289751
dtype: float64

El número de reviews y las reviews per month a priori parece que podrían presentar cierta colinealidad. Además, como reviews_per_month presenta un 29% de Nulls, hemos optado por eliminar la susodicha columna.

In [ ]:
df.drop(['reviews_per_month'], axis = 1, inplace = True)

### Reviews Score

In [ ]:
df[['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
    'review_scores_communication', 'review_scores_location', 'review_scores_value']]

,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,100.0,9.0,10.0,10.0,10.0,10.0,10.0
1,77.0,8.0,9.0,9.0,9.0,9.0,8.0
2,83.0,9.0,9.0,10.0,10.0,9.0,9.0
3,84.0,9.0,9.0,9.0,9.0,8.0,9.0
4,80.0,8.0,8.0,9.0,9.0,9.0,8.0
...,...,...,...,...,...,...,...
75553,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75554,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75555,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75556,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
    'review_scores_communication', 'review_scores_location', 'review_scores_value']].isnull().sum()/df.shape[0]

review_scores_rating           0.304373
review_scores_accuracy         0.305246
review_scores_cleanliness      0.304902
review_scores_checkin          0.305545
review_scores_communication    0.305063
review_scores_location         0.305522
review_scores_value            0.305729
dtype: float64

Al presentar casi 1/3 de valores nulos, y concentrarse la mayoría de los scores entre 9 y 10, eliminamos todas las columnas excepto la que recoge las nota más general del alojamiento, **review_scores_rating**, que procedermos a discretrizar para añadir la categoría **unavaliable**.

In [ ]:
temp = df['review_scores_accuracy'].fillna(9999)
temp.value_counts(dropna = False, normalize = True).sort_index(ascending = False).cumsum()

9999.0    0.305246
10.0      0.679304
9.0       0.900933
8.0       0.967031
7.0       0.981056
6.0       0.992091
5.0       0.993655
4.0       0.996482
3.0       0.996574
2.0       1.000000
Name: review_scores_accuracy, dtype: float64

In [ ]:
temp = df['review_scores_cleanliness'].fillna(9999)
temp.value_counts(dropna = False, normalize = True).sort_index(ascending = False).cumsum()

9999.0    0.304902
10.0      0.645646
9.0       0.865482
8.0       0.953513
7.0       0.974779
6.0       0.989976
5.0       0.992528
4.0       0.996069
3.0       0.996367
2.0       1.000000
Name: review_scores_cleanliness, dtype: float64

In [ ]:
df.drop(['review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'], axis = 1, inplace = True)

In [ ]:
temp = df['review_scores_rating'].fillna(-9999)
temp.value_counts(dropna = False, normalize = True).sort_index(ascending = False).cumsum()

 100.0     0.161210
 99.0      0.170889
 98.0      0.193351
 97.0      0.221561
 96.0      0.253816
 95.0      0.288670
 94.0      0.314558
 93.0      0.358953
 92.0      0.385507
 91.0      0.410452
 90.0      0.458088
 89.0      0.477561
 88.0      0.499540
 87.0      0.524094
 86.0      0.536003
 85.0      0.551016
 84.0      0.564029
 83.0      0.575593
 82.0      0.581732
 81.0      0.585640
 80.0      0.636817
 79.0      0.639300
 78.0      0.643438
 77.0      0.647186
 76.0      0.650841
 75.0      0.654681
 74.0      0.656083
 73.0      0.660704
 72.0      0.662015
 71.0      0.663118
 70.0      0.669280
 69.0      0.669878
 68.0      0.670751
 67.0      0.673165
 66.0      0.673395
 65.0      0.674269
 64.0      0.674821
 63.0      0.675189
 62.0      0.675257
 61.0      0.675280
 60.0      0.686408
 58.0      0.686431
 57.0      0.686477
 56.0      0.686615
 55.0      0.686891
 54.0      0.687006
 53.0      0.687351
 50.0      0.688661
 47.0      0.688868
 46.0      0.688891


La columna **review_scores_rating** ha sido discretizada como se ha dicho antes, en cuatro categorias, **Excellent** en el caso de que la score sea mayor a 90, **Good** en el caso de que esté entre 90 y 70, **NotGood** para las restantes, así como **Unavailable** en caso de no estar disponible.

In [ ]:
df['review_scores_rating'] = df['review_scores_rating'].apply(lambda x: 'Excellent' if x >= 90 else \
                                                              ('Good' if x < 90 and x > 70 else ('NotGood' if x > 0 else 'Unavailable')))

In [ ]:
df['review_scores_rating'].value_counts()

Excellent      19925
Unavailable    13239
Good            8918
NotGood         1414
Name: review_scores_rating, dtype: int64

In [ ]:
df.isnull().sum()[df.isnull().sum()>0]

host_about    17774
dtype: int64

##**Calendar**

**Importamos los datos**

In [ ]:
cal = pd.read_csv("https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Calendar/Calendar_April2016.zip")
cal = cal.append(pd.read_csv("https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Calendar/Calendar_April2017.zip"), ignore_index = True)
cal = cal.append(pd.read_csv("https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Calendar/Calendar_October2018.zip"), ignore_index = True)
cal = cal.append(pd.read_csv("https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Calendar/Calendar_April2018.zip"), ignore_index = True)
cal = cal.append(pd.read_csv("https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Calendar/Calendar_October2018.zip"), ignore_index = True)
cal = cal.append(pd.read_csv("https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Calendar/Calendar_April2019.zip"), ignore_index = True)
cal = cal.append(pd.read_csv("https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Calendar/Calendar_October2019.zip"), ignore_index = True)
cal = cal.append(pd.read_csv("https://github.com/guiruha/TFMAirBNB/raw/master/archivos/Calendar/Calendar_April2020.zip"), ignore_index = True)

Respecto a los datos de calendar nos interesa tanto la media de su precio como se disponibilidad mensual (debido a razones de capacidad de nuestras máquinas y Google Colab nos vemos obligados a tratar datos a nivel mensual y no diario). Por tanto, transformamos la columna date a el tipo **datetime** a fin de sacar el año y el mes como atributos, los cuales también nos serviran de ayuda para captura la muy posible existencia de estacionalidad y tendencia.

In [ ]:
cal = cal[['listing_id', 'date', 'price', 'available']]

In [ ]:
cal[cal['price'].isnull()]['available']

75          f
76          f
77          f
131         f
132         f
           ..
42574259    t
42574260    t
42574261    t
42574262    t
42574263    t
Name: available, Length: 11867462, dtype: object

Antes debemos eliminar posibles duplicados a fin de que cada registro sea único. Como era de esperar existen muchos listings que no entraron en la plataforma años más tarde del inicio de los registros que poseemos, curiosamente esto se ve reflejado en los datasets de **calendar.csv** en forma de nulls (un análisis más en profundidad demuestra que muchos registros no empiezan mostrar valores númericos hasta finales del año 2019), es por ello que filtramos por los precios que no son nulos para evitar problemas en el cálculo de precios medios mensuales más adelante.

In [ ]:
cal = cal[cal['price'].notnull()]

In [ ]:
cal.drop_duplicates(subset = ['date', 'listing_id'], inplace = True)

In [ ]:
cal['date'] = pd.to_datetime(cal['date'])

In [ ]:
cal['month_year'] = cal['date'].dt.to_period('M')
cal['year'] = cal['date'].dt.year
cal['month'] = cal['date'].dt.month

Tras obtener el año y mes de la columna **date**, procedemos a hacer la correspondiente transformación de la columna **availability** para pasarla a variable dicotómica y realizamos los cálculos pertinentes.

In [ ]:
cal['availability'] = cal['available'].map({'t': 1, 'f': 0})

In [ ]:
availability = cal.groupby(['year', 'listing_id'])['availability'].sum()

In [ ]:
availability

year  listing_id
2016  11194          54
      15090          36
      18653          39
      18666          43
      18674          36
                   ... 
2021  43203215        0
      43203651      106
      43203899      106
      43203952      110
      43204355      106
Name: availability, Length: 134728, dtype: int64

A continuación, llevamos a cabo el mismo proceso con la variable **price** (la abordamos igual que las variables de precio del dataset anterior), modificamos de nuevo el dataframe para quedarnos con los precios medios por mes y año de cada listing. No obstante, como queremos evitar que el cálculo resulte en Nulls, deberemos eliminar las columnas con Nulls en la columna **price** una vez calculado el **year_availability**.

In [ ]:
cal['price'] = cal['price'].str.replace('$', '').str.replace(',', '').astype('float')

Un rápido vistazo a como se comporta la variable **available** cuando los precios superan los 9500€ nos muestra que la mayoría de alojamientos se encuentran no disponibles. De este comportamiento hemos deducido que se debe a una asignación a próposito de precios muy elevados por parte del host debido a que no el alojamiento no va a estar disponible durante ese periodo, o bien uno de los comportamientos típicos de los usuarios de AirBNB que explicamos más adelante en Exploración General.

A fin de que estas prácticas no nos suponga ninguna anomalía en el cálculo de precios medios mensuales procederemos a filtrar estos registros dado que no los podemos aprovechar dado nuestro enfoque.

In [ ]:
cal[cal['price']>9500]['year'].value_counts()

2020    63408
2021    58168
2019     3869
2017      436
2018      181
Name: year, dtype: int64

In [ ]:
cal = cal[cal['price']<9500]

In [ ]:
cal = cal.groupby(['month_year','year', 'month', 'listing_id'])['price'].mean().reset_index()

In [ ]:
cal = cal.join(availability, on = ['year', 'listing_id'])

In [ ]:
cal.columns = ['month_year','year', 'month', 'id', 'price_calendar', 'year_availability']

In [ ]:
cal.sample(10)

,month_year,year,month,id,price_calendar,year_availability
641436,2020-05,2020,5,34586395,178.000000,0
655100,2020-06,2020,6,9937338,50.000000,224
333382,2019-04,2019,4,24819159,149.523810,290
190504,2017-11,2017,11,14486701,23.000000,365
235907,2018-04,2018,4,1238107,36.428571,154
729378,2020-09,2020,9,1238141,178.000000,167
577717,2020-03,2020,3,22993193,130.000000,192
888630,2021-04,2021,4,1039162,196.333333,0
512758,2019-12,2019,12,32959971,35.000000,37
628286,2020-05,2020,5,7067195,34.774194,346


##**Dataset Limpio**

Finalmente, considerando que la limpieza del dataset está completada, realizamos un merge entre los dataframes de calendar y listings.


In [ ]:
dfclean = pd.merge(df, cal, how = 'inner', on = 'id')

Al hacer el merge nos percatamos que existen un número relevante de registros con valores bastante superiores a los precios establecidos en los listings. No obstante, un análisis más en profundidad revela que en su gran mayoría los precios coinciden y las diferencias notables se encuentran en los meses donde el alquiler vacacional se encuentra en el pico de demanda (sobretodo los meses de verano).

In [ ]:
print('{} de nulls'.format(dfclean['price_calendar'].isnull().sum()))
print('{} del total'.format(dfclean['price_calendar'].isnull().sum()/dfclean['price_calendar'].shape[0]))

0 de nulls
0.0 del total


In [ ]:
dfclean[['price', 'price_calendar', 'month_year']].sample(20)

,price,price_calendar,month_year
253196,22.0,28.000000,2019-11
740737,65.0,65.000000,2019-12
491508,12.0,25.000000,2019-04
555289,33.0,35.000000,2021-01
604470,89.0,516.700000,2019-06
97639,100.0,100.000000,2020-02
396157,125.0,125.000000,2019-02
162012,150.0,150.000000,2017-08
5803,50.0,50.000000,2019-09
122715,35.0,71.000000,2020-08


Finalmente llamaremos **goodprice** a la variable precio definitiva y eliminaremos las dos columnas de precio producidas por la fusión.

In [ ]:
dfclean['goodprice'] = dfclean['price_calendar']
dfclean.drop(['price', 'price_calendar'], axis = 1, inplace = True)

In [ ]:
dfclean[['month_year', 'year', 'month', 'goodprice', 'year_availability', 'availability_365']].sample(10)

,month_year,year,month,goodprice,year_availability,availability_365
796340,2020-07,2020,7,110.000000,37,71
336611,2017-10,2017,10,30.000000,309,103
35444,2018-02,2018,2,85.000000,97,281
394317,2020-07,2020,7,89.903226,0,359
464160,2019-08,2019,8,195.967742,355,267
142069,2017-10,2017,10,42.000000,133,172
600748,2018-12,2018,12,16.000000,59,180
692889,2019-05,2019,5,25.000000,294,294
348414,2018-04,2018,4,30.000000,97,365
298108,2016-11,2016,11,30.000000,48,70


In [ ]:
print('Nos hemos quedado con un dataframe de {} filas y {} columnas\n'.format(dfclean.shape[0], dfclean.shape[1]))
dfclean.sample(5)

Nos hemos quedado con un dataframe de 815440 filas y 57 columnas



,id,host_since,host_about,host_response_time,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,requires_license,instant_bookable,cancellation_policy,host_emailverified,host_phoneverified,host_hasjumio,host_reviewverified,host_selfieverified,host_idverified,Air conditioning,Family/kid friendly,Host greets you,Laptop friendly workspace,Paid parking off premises,Patio or balcony,Luggage dropoff allowed,Long term stays allowed,Smoking allowed,Step-free access,Pets allowed,24-hour check-in,Elevator,bathrooms_imput,bedrooms_imput,beds_imput,month_year,year,month,year_availability,goodprice
322076,1015758,2011-07-11,.,within an hour,0,3.0,1,1,Gràcia,41.399941,2.162491,1,House,Private room,4,1.0,1.0,1.0,100.0,25.0,2,20.0,1,1125,293,100,Good,t,f,strict_less30,1,1,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,2019-09,2019,9,162,40.000000
668165,33107819,2019-03-14,NaN,within an hour,0,12.0,1,0,Ciutat Vella,41.381550,2.165320,0,Apartment,Private room,3,1.0,1.0,2.0,0.0,0.0,1,0.0,1,1125,1,0,Unavailable,t,t,strict_less30,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2019-04,2019,4,1,66.809524
353312,17429193,2012-12-21,Hi everyone!\r\n\r\nI'm Jacques and I work in ...,within an hour,0,91.0,1,1,Sant Martí,41.395613,2.201631,1,Apartment,Entire home/apt,4,2.0,2.0,3.0,150.0,100.0,1,0.0,3,1125,252,0,Unavailable,t,t,strict_less30,1,1,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,2018-04,2018,4,97,300.000000
28276,8122193,2015-08-30,"HOLA! We are a welcoming, respectful and frien...",within an hour,0,1.0,1,1,Ciutat Vella,41.380533,2.175454,1,Apartment,Private room,3,1.0,1.0,1.0,0.0,0.0,2,8.0,3,15,158,54,Excellent,t,f,strict_less30,1,1,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,2020-11,2020,11,174,52.000000
424162,22351667,2015-06-04,"Passionate traveler, in love with life and goo...",within a few hours,0,5.0,1,0,Ciutat Vella,41.380492,2.185713,1,Other,Entire home/apt,7,1.0,4.0,4.0,0.0,100.0,1,0.0,2,40,363,0,Unavailable,t,t,strict_less30,1,1,1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2020-10,2020,10,288,671.428571


**HASTA AQUÍ LLEGA EL NOTEBOOK/SCRIPT DE LIMPIEZA** El dataframe limpio es transformado en un csv, pickle, o el archivo conveniente a través de la libería pandas para su posterior tratamiento en la **FASE 2: EXPLORACIÓN**